In [7]:
import os

source = '/Users/HQ/orange/Reza/TF/NAR/' #14-two-sided-dummy-contacts/'
systems = sorted([i for i in os.listdir(source) if '.' not in i])
systems = ['1a74', '1azp', '1by4', '1bgb',  '2b0d', '1cdw', '1dh3', '1jj4', '1r4o', '1r4r', '1zme', '2dgc', '2r1j', '3cro', '1ysa', '2dgc']

for system in systems:
    os.chdir(source+system+'/14-two-sided-dummy-contacts/')
    rmsd = open('rmsd00.cpptraj', 'r').readlines()
    mask = ','.join([str(x) for x in sorted([int(j) for j in [i for i in rmsd if 'rms traj00 ref [ref] :' in i][0].split(':')[1].split('@')[0].split(',')])])
    with open('clusters-rmsd.sh', 'w') as outfile:
        outfile.write('''#!/bin/bash

cpptraj topol.top<<EOF
trajin trajectory.00.dcd  10000 10001
strip :SDM parmout topol-no-dummy.top
go
EOF

# PD

cpptraj<<EOF
parm topol-no-dummy.top [system]
parm reference.prmtop [reftop]
reference reference.inpcrd parm [reftop] [ref]
trajin contact-clusters-single-traj-pd/DNA-PROT/unique.c?.pdb parm [system]

align :%s@N,C,CA,O5',C4',C1' ref [ref]
rms top10-pd ref [ref] :%s@N,C,CA,O5',C4',C1' nofit out rmsd-top10-pd.dat
go
EOF

# Normal

cpptraj<<EOF
parm topol-no-dummy.top [system]
parm reference.prmtop [reftop]
reference reference.inpcrd parm [reftop] [ref]
trajin contact-clusters-single-traj/DNA-PROT/unique.c?.pdb parm [system]

align @O5',C4',C1' ref [ref]
rms fit @O5',C4',C1'
rms top10 ref [ref] :%s@N,C,CA nofit out rmsd-top10.dat
go
EOF

# LI

cpptraj<<EOF
parm topol-no-dummy.top [system]
parm reference.prmtop [reftop]
reference reference.inpcrd parm [reftop] [ref]
trajin contact-clusters-single-traj-li/DNA/unique.c?.pdb parm [system]

align :%s@N,C,CA ref [ref]
rms fit ref [ref] :%s@N,C,CA
rms top10-li ref [ref] :%s@O5',C4',C1' nofit out rmsd-top10-li.dat
go
EOF
'''%(mask, mask, mask, mask, mask, mask))